### 1. What is the purpose of this experiment?
LGBMを使ってWordNetで取得したlemmaの選別を行う
### 2. Why do you this?
kernelではlemma全てを処理しきれないため。また、不要なlemmmaも数多く含まれているため
### 3. Where are the points of technology and techniques?
LGBM, TFIDF
### 4. How do you validate the effectiveness?
処理時間
CV
### 5. What will you do next?
選別したlemmaをkerasに入れて精度と計算速度を見る

In [1]:
%load_ext autoreload
%autoreload 2
feat_no = 102
is_tfidf = True
is_svd = True
from main_quara import quara_load_data, cleansing_text
from tqdm import tqdm
import numpy as np
import pandas as pd
import time
import datetime
import sys
import re
import gc
import glob
import pickle as pkl
import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, pararell_process
logger = logger_func()

from joblib import Parallel, delayed
from contextlib import contextmanager
@contextmanager
def timer(name):
    """
    Taken from Konstantin Lopuhin https://www.kaggle.com/lopuhin
    in script named : Mercari Golf: 0.3875 CV in 75 LOC, 1900 s
    https://www.kaggle.com/lopuhin/mercari-golf-0-3875-cv-in-75-loc-1900-s
    """
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')

# NLP
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

key = 'qid'
qt = 'question_text'
target = 'target'

2018-11-18 20:19:51,314 utils 353 [INFO]    [logger_func] start 


In [2]:
from nltk.corpus import wordnet as wn
is_lemma=True
is_original=False
logger.info("Load Data...")
train, test = quara_load_data()

# Load id Text List
train_id_list = list(train[key].values)
test_id_list = list(test[key].values)
train_text_list = list(train[qt].values)
test_text_list = list(test[qt].values)
id_list = train_id_list + test_id_list
text_list = train_text_list + test_text_list

with timer("Cleansing Text..."):
    # 並列処理でクレンジング
    train_text_list = Parallel(n_jobs=-1)( [delayed(cleansing_text)(args) for args in train_text_list] )
    test_text_list = Parallel(n_jobs=-1)( [delayed(cleansing_text)(args) for args in test_text_list] )
#     train_text_list = pararell_process(pararell_cleansing, train_text_list)
#     test_text_list = pararell_process(pararell_cleansing, test_text_list)
    text_list = train_text_list + test_text_list
    
if is_lemma:
    #========================================================================
    # Get lemma List
    #========================================================================
    def get_wordnet_lemma(word):
        lemma_list = []
        for w in wn.synsets(word):
            lemma_list+=w.lemma_names()
        return list(set(lemma_list) - set([word]))
    
    
    # まずは全てのlemmaを取得する
    def pararell_get_lemma(args):
        uid = args[0]
        doc = args[1]
        word_list = doc.split()
        tmp_dict = {}
        lemma_list = []
        for word in word_list:
            lemma_list += get_wordnet_lemma(word)
        lemma = " ".join(lemma_list)
        lemma = cleansing_text(lemma)
        if is_original:
            tmp_dict[uid] = doc + lemma
        else:
            tmp_dict[uid] = lemma
        return tmp_dict
            
    with timer("Get lemma from Wordnet"):
        p_list = pararell_process(pararell_get_lemma, zip(id_list, text_list))
    with timer("Make lemma dataset"):
        lemma_dict = {}
        [lemma_dict.update(p) for p in p_list]

2018-11-18 20:19:54,177 utils 4 [INFO]    [<module>] Load Data... 
100%|██████████| 3/3 [00:00<00:00, 178.33it/s]


[Cleansing Text...] done in 35 s
[Get lemma from Wordnet] done in 116 s
[Make lemma dataset] done in 0 s


In [5]:
#========================================================================
# TFIDF
#========================================================================
def get_tfidf(text_list):
    '''
    Explain:
        テキストを1要素としてもったリストを渡し、TFIDFベクトルをもったsparse matrixを出力する
    Args:
        text_list(list): split前のテキストリスト. 1テキストが1つのTFIDFベクトルに変換される
    Return:
        sparse csr_matrix: TFIDF値が入ったスパースな行列
    '''
    # Get the tfidf
    with timer("Fitting TFIDF"):
        # TFIDFに変換するオブジェクトを作成する
        tfidf_vectorizer = TfidfVectorizer(
            max_features = 30000,
            min_df=10,
            max_df=0.5,
            stop_words="english",
            analyzer='word',
            #  analyzer='char',
            strip_accents='unicode',
            ngram_range=(1,1),
            use_idf=True,
            smooth_idf=True,
            sublinear_tf=True
        ).fit(text_list)

    # Train, Testのテキストが順番に並んだリストを渡すと、各テキストをTFIDFベクトルに変換して返してくれる
    # テキストに対応するインデックスを渡して返ってくるのが対応するTFIDFベクトル
    with timer("Transform TFIDF"):
        csr_tfidf = tfidf_vectorizer.transform(text_list)
    with timer("Save TFIDF Matrix"):
        utils.to_pkl_gzip(obj=csr_tfidf, path='./csr_tfidf')
    return csr_tfidf, tfidf_vectorizer

TFIDF
key_list = lemma_dict.keys()
csr_tfidf, tfidf_vectorizer = get_tfidf(lemma_dict.values())
del lemma_dict
gc.collect()
utils.to_pkl_gzip(obj=csr_tfidf, path='../input/csr_tfidf30000.gz')

TFIDFのsparse_csr_matrixをLGBMに入力する前に、結果として得られるfeature_importanceにおいてどの単語のGainが大きかった見れるようにしておく必要がある。
しかし、50,000カラムのsparse_csr_matrixをtoarrayするとそれだけでallocate_memoryとなる。
よって、まずはmatrixのインデックスに対応する各単語をvectorizer.vocabrary_ or vectorizer.get_feature_names()で取得する。
その後、LGBMのfeature_importance()がカラム順で出力されるのを確認した上で、対応づける。
なお、50,000カラムではLGBMの学習の時間がかかりすぎるので、5,000~10,000カラムに分割し、それぞれLGBMにかけて単語を選別していく。

In [40]:
st_time = time.time()
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import log_loss, confusion_matrix, f1_score, accuracy_score
sys.path.append(f'{HOME}/kaggle/data_analysis/model')
from params_lgbm import params_quara
from xray_wrapper import Xray_Cal
sys.path.append(f'{HOME}/kaggle/data_analysis')
from model.lightgbm_ex import lightgbm_ex as lgb_ex
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

# Args
feature_list = np.array(tfidf_vectorizer.get_feature_names())
feat_split_size = 4

# LGBM Args
model_type = 'lgb'
fold_type = 'stratified'
metric = 'accuracy'
fold = 2
learning_rate = 0.1
early_stopping_rounds = 100
num_boost_round = 3000
seed = 1208
params = params_quara()
params['device'] = 'gpu'

# Result
select_features = [] # Feature Importanceによって選択したfeature群を入れるリスト

with timer("LGBM Setting"):
    
    y = train[target]
    prediction = np.array([])
    
    # testも結合されてるので、trainのみにする
    tmp = csr_tfidf[list(train.index)]
    
    # 一度に数万カラムの学習は難しいので分割する
    feat_idx = np.arange(len(feature_list))
    np.random.seed(seed)
    np.random.shuffle(feat_idx)
    idx_list = np.split(feat_idx, feat_split_size)
    
    
# 事前に分割したfeatureのindexのみでそれぞれ学習を行い、結果を出力する
for idx_no, idx in enumerate(idx_list):
    tmp_train = tmp.T[idx].T

    ' KFold '
    if fold_type == 'stratified':
        folds = StratifiedKFold(n_splits=fold, shuffle=True, random_state=seed)
        kfold = folds.split(tmp_train, y)
    
    cv_feim = pd.DataFrame() # Feature Importanceの結果ファイルを入れるDF
    for n_fold, (trn_idx, val_idx) in enumerate(kfold):
        
        with timer(f"Validation: {n_fold} | LGBM Train"):
            x_train, y_train = tmp_train[trn_idx], y[trn_idx]
            x_val, y_val = tmp_train[val_idx], y[val_idx]
        
            # Dataset
            lgb_train = lgb.Dataset(data=x_train, label=y_train)
            lgb_eval = lgb.Dataset(data=x_val, label=y_val)
            
            estimator = lgb.train(
                train_set=lgb_train,
                valid_sets=lgb_eval,
                params=params,
                verbose_eval=200,
                early_stopping_rounds=early_stopping_rounds,
                num_boost_round=num_boost_round
            )
        
        with timer(f"Validation: {n_fold} | Prediction & Get F1 score"):
            y_pred = estimator.predict(x_val)
            score = log_loss(y_val, y_pred)
            logger.info(f'Fold No: {n_fold} | {metric}: {score}')
            logger.info(f"Train Shape: {x_train.shape}")
            for thresh in np.arange(0.1, 0.301, 0.01):
                thresh = np.round(thresh, 2)
                f1 = f1_score(y_val, (y_pred>thresh).astype(int))
                logger.info(f"F1 score at threshold {thresh} is {f1}")
                
            ' Feature Importance '
            if len(cv_feim):
                cv_feim[f'{n_fold}_importance'] = estimator.feature_importance(importance_type='gain')
            else:
                feim_name = f'{n_fold}_importance'
                feim = pd.Series(estimator.feature_importance(importance_type='gain'), name=feim_name, index=feature_list[idx_list[0]]).to_frame().reset_index().rename(columns={'index':'feature'})
                cv_feim = feim.copy()
                
        
    with timer("Save Feature Importance"):
        col_feim = [col for col in cv_feim.columns if col.count('importance')]
        cv_feim['avg_importance'] = cv_feim[col_feim].mean(axis=1)
        cv_feim.sort_values(by='avg_importance', ascending=False, inplace=True)
        from matplotlib import pyplot as plt
        import japanize_matplotlib
        %matplotlib inline
        import seaborn as sns
        plt.figure(figsize=(8, 12))
        display(cv_feim.head())
        sns.barplot(data=cv_feim.iloc[:50, ], x='avg_importance', y='feature')
        plt.show()
        cv_feim.to_csv(f'../valid/{start_time[4:12]}_{model_type}_TFIDF{idx_no}_f1{f1}_logloss{score}_lr{learning_rate}.csv', index=False)
        tmp_features = list(cv_feim[cv_feim['avg_importance']>30]['feature'].values)
        print(f'Selece Feature: {len(tmp_features)}')
        select_features += tmp_features
        
select_features = list(set(select_features))
print(len(select_features))
print(sorted(select_features))
print(f'All done in {time.time() - st_time:.0f} s')

[array(['muslim', 27386.81890198472, 33036.2978767911, 30211.55838938791],
      dtype=object), array(['female', 30389.32406795211, 26010.4733360901, 28199.898702021106],
      dtype=object), array(['citizenry', 27985.749314332526, 18079.7445953195,
       23032.746954826012], dtype=object), array(['liberalist', 18264.89389471989, 20417.157704297453,
       19341.025799508672], dtype=object), array(['red', 17004.713838519296, 15576.036513215746, 16290.37517586752],
      dtype=object), array(['mass', 7663.214119587553, 15141.594242088067, 11402.40418083781],
      dtype=object), array(['israelite', 10325.834320679307, 10605.147466171533,
       10465.49089342542], dtype=object), array(['fair', 7959.968800886534, 8734.757852984942, 8347.363326935738],
      dtype=object), array(['vote', 7672.977117855102, 5649.157958668657, 6661.067538261879],
      dtype=object), array(['sexuality', 5198.632705069613, 7303.273845876916,
       6250.9532754732645], dtype=object)]


SystemExit: 

Feature Importanceにより絞られたlemma_featureのみのsparse_matrixを作成する.  
ここで出力されたcsr_matrixが、当初の目的だったFeature Importanceにより上位選択されたlemmaのデータセットとなる.

In [76]:
id_list = []
[id_list.append(id)  for word, id in tfidf_vectorizer.vocabulary_.items() if word in select_features]
print(len(id_list))
print(len(select_features))
tmp_train = csr_tfidf.T[id_list].T
tmp_train.shape

1851
1851


(1362492, 1851)

In [73]:
#========================================================================
# Make Train Validation
# Tokenizer
#========================================================================

## some config values 
embed_size = 200 # how big is each word vector
max_features = len(id_list) # how many unique words to use (i.e num rows in embedding vector)
maxlen = len(id_list) # max number of words in a question to use

with timer("Make Train Vaidation Set & Tokenizer"):
    
    ## split to train and val
    train_X, val_X = train_test_split(train, test_size=0.2, random_state=seed)
    trn_idx, val_idx = list(train_X.index), list(val_X.index)
    
    train_X, val_X = tmp_train[trn_idx], tmp_train[val_idx]
    ## Get the target values
    train_y = train['target'].values[trn_idx]
    val_y = train['target'].values[val_idx]

[Make Train Vaidation Set & Tokenizer] done in 1 s


In [78]:
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import tensorflow as tf
print(tf.test.is_built_with_cuda())

#========================================================================
# No PreTrain Model
#========================================================================
def no_pretrain_NN():
    with timer("Create No PreTrain Model"):
        inp = Input(shape=(maxlen,))
        x = Embedding(max_features, embed_size)(inp)
#         x = Bidirectional(CuDNNLSTM(64, return_sequences=True))(x)
        x = Bidirectional(LSTM(64, return_sequences=True))(x)
        x = GlobalMaxPool1D()(x)
        x = Dense(16, activation="relu")(x)
        x = Dropout(0.1)(x)
        x = Dense(1, activation="sigmoid")(x)
        model = Model(inputs=inp, outputs=x)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        
        print(model.summary())
        
    with timer("Model Fitting"):
        ## Train the model 
        model.fit(train_X, train_y, batch_size=512, epochs=2,
                  validation_data=(val_X, val_y)
                 )
        
    with timer("Prediction & Get F1 score"):
        pred_noemb_val_y = model.predict([val_X], batch_size=1024, verbose=1)
        for thresh in np.arange(0.1, 0.501, 0.01):
            thresh = np.round(thresh, 2)
            print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_noemb_val_y>thresh).astype(int))))
        del model, inp, x
        import gc; gc.collect()
        time.sleep(10)
with timer(f"Done No PreTrain Model"):
    no_pretrain_NN()

True
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 1851)              0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 1851, 300)         555300    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 1851, 128)         186880    
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 17        
Total

InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' with these attrs.  Registered devices: [CPU,XLA_CPU,XLA_GPU], Registered kernels:
  device='GPU'; T in [DT_DOUBLE]
  device='GPU'; T in [DT_FLOAT]
  device='GPU'; T in [DT_HALF]

	 [[node bidirectional_1/CudnnRNN (defined at /home/ubuntu/anaconda3/lib/python3.6/site-packages/tensorflow/contrib/cudnn_rnn/python/ops/cudnn_rnn_ops.py:922)  = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="lstm", seed=87654321, seed2=0](bidirectional_1/transpose, bidirectional_1/ExpandDims_1, bidirectional_1/ExpandDims_2, bidirectional_1/concat)]]

Caused by op 'bidirectional_1/CudnnRNN', defined at:
  File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/lib/python3.6/site-p

In [5]:
#========================================================================
# Glove PreTrain Model
#========================================================================
def glove_pretrain_NN():
    with timer("Get Glove PreTrain Grad"):
        EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
        def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
        embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))
        
        all_embs = np.stack(embeddings_index.values())
        emb_mean,emb_std = all_embs.mean(), all_embs.std()
        embed_size = all_embs.shape[1]
        
        word_index = tokenizer.word_index
        nb_words = min(max_features, len(word_index))
        embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
        for word, i in word_index.items():
            if i >= max_features: continue
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    with timer("Create Glove PreTrain Model"):
        inp = Input(shape=(maxlen,))
        x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
        x = Bidirectional(CuDNNGRU(64, return_sequences=True))(x)
        x = GlobalMaxPool1D()(x)
        x = Dense(16, activation="relu")(x)
        x = Dropout(0.1)(x)
        x = Dense(1, activation="sigmoid")(x)
        model = Model(inputs=inp, outputs=x)
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
        print(model.summary())
        
    with timer("Model Fitting"):
        model.fit(train_X, train_y, batch_size=512, epochs=2, validation_data=(val_X, val_y))
        
    with timer("Prediction & Get F1 score"):
        pred_glove_val_y = model.predict([val_X], batch_size=1024, verbose=1)
        for thresh in np.arange(0.1, 0.501, 0.01):
            thresh = np.round(thresh, 2)
            print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 300)          9000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          140544    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                2064      
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total para

In [ ]:
with timer("Make Train Vaidation Set & Tokenizer"):
    
    ## fill up the missing values
    train_X = tmp_train["question_text"].fillna("_na_").values
    
    ## Tokenize the sentences
    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(train_X))
    train_X = tokenizer.texts_to_sequences(train_X)
    # test_X = tokenizer.texts_to_sequences(test_X)
    
    ## Pad the sentences 
    train_X = pad_sequences(train_X, maxlen=maxlen)
    # test_X = pad_sequences(test_X, maxlen=maxlen)
    
    ## Get the target values
    train_y = train_df['target'].values

    # KFold
    from sklearn.model_selection import StratifiedKFold
    if fold_type == 'stratified':
        folds = StratifiedKFold(n_splits=fold, shuffle=True, random_state=seed)  # 1
        kfold = folds.split(train_X, train_y)

    for n_fold, (trn_idx, val_idx) in enumerate(kfold):
        x_train, x_val = train_X[train_idx], train_X[val_idx]
        y_train, y_val = train_y[train_idx], train_y[val_idx] 